## **A/B Testing**<br>
- - - 

#### **Info on datasets**:
You can find the dataset &rarr; [here](https://www.kaggle.com/datasets/zhangluyuan/ab-testing?select=ab_data.csv).
- The dataset contains information about almost 300K users that were involved in a A/B test. It is an **`unpaired`** dataset
- Features:
    - user_id: unique identifier for each user.
    - timestamp: associated date and time for each visit to the website by a given user.
    - group: the category a user was grouped into pre-A/B test (control or treatment groups).
    - landing_page: the page that was displayed to a user when they visited the company website (new_page or old_page).
    - converted: whether a user converted or not (0 or 1); Note: Users in the control group ought to be displayed the old page, while those in the treatment group ought to see the new page.
    - - - 

###  &rarr; **Preprocessing data** 
- Note: **unpaired dataset**.<br>


#### **Importing libraries**:

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
from statsmodels.stats.proportion import proportions_ztest
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#### **Read the dataset**:

In [2]:
df = pd.read_csv('s3://demo-sagemaker-bucket-2222/ab_data.csv', sep=",", header=0)

df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [3]:
df.shape

(294478, 5)

#### **To make sure the whole control group is seeing the old page and the whole treatment group is seeing the new page**:

In [4]:
indexes_1 = df[ (df['group'] == 'control') & (df['landing_page'] == 'new_page') ].index
 
df.drop(indexes_1,inplace=True)

In [5]:
indexes_2 = df[ (df['group'] == 'treatment') & (df['landing_page'] == 'old_page') ].index
 
df.drop(indexes_2,inplace=True)

In [6]:
df.shape

(290585, 5)

- **Note**: **3893** times that either old_page and control or new_page and treatment are **not aligned**

#### **Remove the one duplicated `user_id`**:

In [7]:
df.user_id.nunique()

290584

In [8]:
df.drop_duplicates(subset='user_id', inplace=True)

In [9]:
len(df)

290584

In [10]:
df.reset_index(drop=True, inplace=True)
df

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
...,...,...,...,...,...
290579,751197,2017-01-03 22:28:38.630509,control,old_page,0
290580,945152,2017-01-12 00:51:57.078372,control,old_page,0
290581,734608,2017-01-22 11:45:03.439544,control,old_page,0
290582,697314,2017-01-15 01:20:28.957438,control,old_page,0


- - - 

###  &rarr; **Two proportions z-test** <br>


### **`Assumptions`**:
- The data are simple **random values** from **both populations**.
- Samples are **independent** of each other.
- Sample **sizes** are **large enough to be approximately normally distributed**. A good **rule of thumb** is that, given the **sample sizes** ***$n_{1}$*** and ***$n_{2}$*** and given the **samples proportions** of ***$p_{1}$*** and ***$p_{2}$*** then **all of the below must be at least 10**:
  > - ***$n_{1}$*** *   ***$p_{1}$***
  - ***$n_{1}$*** *    ***$(1 - p_{1})$***
  - ***$n_{2}$*** *   ***$p_{2}$***
  - ***$n_{2}$*** *    ***$(1 - p_{2})$***
<br>


### **`Hypothesis`**:
- **Null hypothesis**: The difference between population proportions of conversion is equal to zero &rarr; ***$H_{0}$***: **prop_c = prop_t**

- **Alternative hypothesis**: The difference between population proportions of conversion is different from zero &rarr; ***$H_{1}$***: **prop_c ≠ prop_t** (two -tailed) 
- **prop_c** and **prop_t** stand for the **conversion rate** of the **control** and **treatment populations**, respectively.
- - - 
<br>



#### Some **previous calculations** for **computing z-statistic**:

In [11]:
prop_c=df.query("group=='control'")['converted'].mean()
print('Proportion of users converted in the control group : ',prop_c)

Proportion of users converted in the control group :  0.1203863045004612


In [12]:
prop_t=df.query("group=='treatment'")['converted'].mean()
print('Proportion of users converted in the treatment group : ',prop_t)

Proportion of users converted in the treatment group :  0.11880806551510564


In [13]:
num_control=len(df.query("group=='control'"))
num_control

145274

In [14]:
num_treat=len(df.query("group=='treatment'"))
num_treat

145310


- **Calculate the pooled sample proportion**:



In [15]:
p = df.converted.mean()
print('Pooloed sample proportion: ', p)

Pooloed sample proportion:  0.11959708724499628


##### **<center>or</center>**

In [16]:
p = (prop_c * num_control + prop_t * num_treat) / (num_control + num_treat)
print(f'p = {p:.17f}')

p = 0.11959708724499628


- - - 

### &rarr; **Compute `critical z‐value`** and **`z-statistic`**:<br>

#### **1. Critical z-value**:

In [17]:
alpha=0.05
z_critical = stats.norm.ppf(1-alpha/2)
print(f'Critical z-value = {z_critical:.16f}\n')

Critical z-value = 1.9599639845400540



#### **2. Z-statistic**:

#### **2.1 -** The **test statistic is** a **z-score defined** by the **following equation**:



In [18]:
z_score = abs((prop_t - prop_c)/np.sqrt(p*(1-p)*(1/num_treat + 1/num_control)))
print(f'z-score = {z_score:.16f}')

z-score = 1.3109241984234394


In [19]:
if z_score > z_critical:
    print('There is statistical evidence to reject null hypothesis.')
else:
    print ('There is not statistical evidence to reject null hypothesis.')

There is not statistical evidence to reject null hypothesis.


#### **2.2 - Make use** of    **`proportions_ztest() function`** from **statsmodels library** to find **z-statistic**:

In [20]:
from statsmodels.stats.proportion import proportions_ztest

control_results = df[df['group'] == 'control']['converted']
treatment_results = df[df['group'] == 'treatment']['converted']
n_control = control_results.count()
n_treatment = treatment_results.count()
successes = [control_results.sum(), treatment_results.sum()]
n_observations = [n_control, n_treatment]

z_statistic, p_value = proportions_ztest(count=successes, nobs=n_observations, alternative='two-sided')

print(f'z-statistic: {z_statistic:.16f}\n')
print(f'p-value: {p_value:.17f}\n')

if p_value < alpha:
    print('There is statistical evidence to reject null hypothesis.')
else:
    print ('There is not statistical evidence to reject null hypothesis.')


z-statistic: 1.3109241984234394

p-value: 0.18988337448195103

There is not statistical evidence to reject null hypothesis.


- - - 

###  &rarr; **Conclusion**:

- **There is no statistical evidence** that the **populations proportions of conversion** (control and treatment populations) **are different**.